In [5]:
PRINT_TRESHOLD_LEVEL = 1

def print_if_needed(str_to_print: str, message_level=0):
    """Conditional pront function for debugging

    Args:
        str_to_print (str): The printed message
        message_level (int, optional): The debugging-verbosity level. Defaults to 0.
    """
    if (message_level >= PRINT_TRESHOLD_LEVEL):
        print(str_to_print)

# Initial security considerations for using persistent data
### With examples based on Python and SQLite

Goals to accomplish:
1. Access the credentials database
2. Get user input and verify credentials
3. After a predetermined number of guesses, lock the account
4. Contain all the problems without leaking them out

### Querying database (using SQLite3) and exception handling in Python

In [29]:
import sqlite3
import traceback
import sys

db = sqlite3.connect("./test_data.dbf.db")
cursor = db.cursor() 
query = "SELECT * from USERS_TOPIC_1"     
rows = cursor.execute(query).fetchall()
# Loop over rows and print them
for row in rows:
    print_if_needed(row) 

db.commit()
db.close()

## Improve the database operations to make them more resistant to attack
1. ### Handle unexpected issues with the database using exceptions
2. ### Add pretty print function to print out table-shaped data


In [8]:
import sqlite3

db_name = "test_data.dbf.db"

def pretty_print_rows(cursor, rows, ) -> tuple:
    """ Pretty-Prints the Cursor with header
    Args:
        cursor (SQLite3 Cursor): DB-Cursor
        rows (SQLite3 Data Rows): Data-Rows
    """

    user_profile = (-1, '')
    print_if_needed(f"Rows:\n{rows}")
    if len(rows)>0: # Do work only when there are returned data records
        # Unpack, Format, and Print Column-Header Descriptions 
        # Using Python List-Comprehension
        headers = [description[0] for description in cursor.description]
        row_header = '\t'
        for col_name in headers:
            row_header += f' \t {col_name} '
        head_line = f'{row_header}\t'
        p = len(head_line)
        line = '\t  '+'_'*(p+3*len(headers)+1) # compensate tabs' width
        print_if_needed(f'{line}\n{head_line}\n{line}')

        for row in rows:        
            row_string = '\t'
            for entry in row:
                row_string += f' \t {entry} '
            print_if_needed(f'{row_string}\t')

    if len(rows)==1:
        print_if_needed(f" Rows: {rows[0]}")
        user_profile = tuple([rows[0][0],rows[0][1]])

    return user_profile
#----------------------------------------------------------------------------

def execute_query(db_file_name:str, user:str, pwd:str) -> tuple:
    returned_rows_count = -1 # Set the initial value negative, as 0 rows is always possible.
    user_profile = (-1, '')
    auth_table = 'USERS_TOPIC_1'
    try:
        db = sqlite3.connect(db_file_name)
        cursor = db.cursor() 
        sql_query = (
                        f""" SELECT * from {auth_table}   """
                        f""" WHERE user_name= ?       """ 
                        f""" AND password_text= ?      """
                    )
        print_if_needed(f"\nThe query constructed:\n\t{sql_query}")
        # sql_data = ("John", "MacDonald")    
        rows = cursor.execute(sql_query, (user, pwd)).fetchall()
        user_profile = pretty_print_rows(cursor, rows)
        returned_rows_count = len(rows) if rows else -10

    except sqlite3.Error as er:
        print_if_needed(f'SQLite3 error: {er.args}')
        print_if_needed(f'Exception class is: {er.__class__}')
        print_if_needed('SQLite traceback: ')
        error_type, error_value, error_trace_back = sys.exc_info()
        print(traceback.format_exception(error_type, error_value, error_trace_back))

    except Exception as ex:
        print(f'Non-SQLite3 Exceptions [{ex}]')

    finally:     
        print_if_needed('Finally! - COMMIT CHANGES to DB (if ANY) and CLOSE DB' )
        db.commit()
        db.close()
        return (user_profile[0], user_profile[1], returned_rows_count)
#----------------------------------------------------------------------------


print(f"\n\n\tfor User:dummy-1 Rows Back: {execute_query(db_name, 'dummy-1', '123456')}\n")

print(f"\n\n\tfor User:dummy-25 Rows Back: {execute_query(db_name, 'dummy-25', 'michael')}\n")

print(f"\n\n\tfor User:dummy-50 Rows Back: {execute_query(db_name, 'dummy-50', 'iloveyou')}\n")

print(f"\n\n\tfor User:dummy-0 Rows Back: {execute_query(db_name, 'dummy-0', 'iloveyou')}\n")

print(f"\n\n\tfor User:dummy-5 Rows Back: {execute_query(db_name, 'dummy-5', 'iloveyou')}\n")




	for User:dummy-1 Rows Back: (1, 'dummy-1', 1)



	for User:dummy-25 Rows Back: (25, 'dummy-25', 1)



	for User:dummy-50 Rows Back: (50, 'dummy-50', 1)



	for User:dummy-0 Rows Back: (-1, '', -10)



	for User:dummy-5 Rows Back: (-1, '', -10)



## Move on to an actual system mock-up AND THINK about the following concepts:

1. #### We MUST NOT expose the system inner workings to the end-user, but why?
- most end-users don't NEED to know how the system works
- most end-users feel BURDENED by unnecessary details at login - from their perspective, the application is a tool to use, not something to work on
- the users who do understand and want to know how systems work COULD BE A POTENTIAL DANGER to the system
- we must keep our system resistant to the efforts of CURIOUS (innocent until proven guilty) users' interference
    
2. #### What happens if user input is not what we expected?
- what if username/password is TOO SHORT/LONG or contains symbols we were not ready to handle?
- do we tell users what symbols are ALLOWED/NOT ALLOWED in the username or password?
- should we restrict WHAT CHARACTERS CAN/CANNOT be PRESENT in the username/password?
- do we restrict user input LENGTH for all the input fields?
- do we count the number of times the user failed to provide correct credentials?
- what do we do when the username does not exist in the database?
- what do we do when the username does not match the expected allowed usernames in the database?
- what do we do if the username exists, but the password is wrong?  

3. #### Can we always trust our own personnel?
- what happens if a database administrator or a developer gets ahold of someone's password?
- what if somebody gets a copy/backup of the login database?
- will the database management and developers always behave appropriately, even in the face of personal threats?

4. #### Consider the lifetime of the application and potential future changes
- what will happen to your code if the database keeps the same data, but the format changes? 
- what will happen if the table name changes in the database?
- how will you solve the problems when an unexpected change breaks the whole system?


In [37]:
LOG_LEVEL = -1
def get_print_user_profile(cursor, rows, ) -> dict:
    """ Pretty-Prints the Cursor with header
    Args:
        cursor (SQLite3 Cursor): DB-Cursor
        rows (SQLite3 Data Rows): Data-Rows
    """
    user_profile = {"id":-10, "name": '', "failed": -10}
    print(f"Rows:\n{rows}")
    if len(rows)>0: # Do work only when there are returned data records
        # Unpack, Format, and Print Column-Header Descriptions 
        # Using Python List-Comprehension
        headers = [description[0] for description in cursor.description]
        row_header = '\t'
        for col_name in headers:
            row_header += f' \t {col_name} '
        head_line = f'{row_header}\t'
        p = len(head_line)
        line = '\t  '+'_'*(p+3*len(headers)+1) # compensate tabs' width
        print(f'{line}\n{head_line}\n{line}')

        for row in rows:        
            row_string = '\t'
            for entry in row:
                row_string += f' \t {entry} '
            print(f'{row_string}\t')

    if len(rows)==1:
        print_if_needed(f" Rows: {rows[0]}")
        row = rows[0]
        user_profile = {"id":row[0], "name": row[1], "failed": row[2]}

    return user_profile
#----------------------------------------------------------------------------
def update_failed_attempts(db_file_name:str, login_table:str, user:str, new_value = -1) -> int:
    try:
        id = -1
        db = sqlite3.connect(db_file_name)
        cursor = db.cursor() 
        sql_query = ( # Instead of * we use EXPLICITLY NAMED columns
                      # Also, we DO NOT PULL BACK PASSWORD data
                        f" UPDATE OR IGNORE [{login_table}]     "
                        f"  SET failed_count = failed_count+1   "
                        f"  WHERE user_name= ?            "
                    ) if new_value<0 else (
                        f" UPDATE OR IGNORE [{login_table}] "
                        f"  SET failed_count = :value       "
                        f"  WHERE user_name= :user          "
                    ) # Both ways of
        param_info = (user,) if new_value<0 else { 'user': f'{user}', 'value': f'{new_value}',}
        print(f"\nThe query constructed:\n\t{sql_query}\n")
        # sql_data = ("John", "MacDonald")    
        id = cursor.execute(sql_query, param_info ).lastrowid
    except sqlite3.Error as er:
        print_if_needed(f'SQLite3 error: {er.args}')
        print_if_needed(f'Exception class is: {er.__class__}')
        print_if_needed('SQLite traceback: ')
        exc_type, exc_value, exc_tb = sys.exc_info()
        print(traceback.format_exception(exc_type, exc_value, exc_tb))

    except Exception as ex:
        print(f'Non-SQLite3 Exceptions [{ex}]')

    finally:     
        print_if_needed('Finally! - COMMIT CHANGES to DB (if ANY) and CLOSE DB' )
        if db:
            db.commit()
            db.close()
        return id

#----------------------------------------------------------------------------

def run_login_query(db_file_name:str, login_table:str, user:str, pwd:str) -> tuple:
    returned_rows_count = -1 # Set the initial value negative, as 0 rows is always possible.
    user_profile = {"id":-1, "name": '', "failed": -1}
    try:
        db = sqlite3.connect(db_file_name)
        cursor = db.cursor() 
        sql_query = ( # Instead of * we use EXPLICITLY NAMED columns
                      # Also, we DO NOT PULL BACK PASSWORD data
                        f" SELECT  id, user_name, failed_count "
                        f"    FROM [{login_table}]               "
                        f"    WHERE user_name='{user}'         "
                        f"    AND password_text='{pwd}'        "
                        f"    AND failed_count <= 10            "
                    )
        print_if_needed(f"\nThe query constructed:\n\t{sql_query}")
        # sql_data = ("John", "MacDonald")    
        rows = cursor.execute(sql_query).fetchall()
        user_profile = get_print_user_profile(cursor, rows)
        returned_rows_count = len(rows) if rows else -10

    except sqlite3.Error as er:
        print_if_needed(f'SQLite3 error: {er.args}')
        print_if_needed(f'Exception class is: {er.__class__}')
        print_if_needed('SQLite traceback: ')
        exc_type, exc_value, exc_tb = sys.exc_info()
        print(traceback.format_exception(exc_type, exc_value, exc_tb))

    except Exception as ex:
        print(f'Non-SQLite3 Exceptions [{ex}]')

    finally:     
        print_if_needed('Finally! - COMMIT CHANGES to DB (if ANY) and CLOSE DB' )
        db.commit()
        db.close()
        if  user_profile['id']<0: # user_profile['failed']>10 or 
            update_failed_attempts(db_file_name, login_table, user)
        else:
            update_failed_attempts(db_file_name, login_table, user, 0)
        return user_profile
#----------------------------------------------------------------------------


db_name = "test_data.dbf.db"
table_name = "Users_Topic_1a"

user_profile_1 = run_login_query(db_name, table_name, 'dummy-1', '123456')
user_profile_10 = run_login_query(db_name, table_name, 'dummy-10', 'dragon')
user_profile_20 = run_login_query(db_name, table_name, 'dummy-20', '666666')
user_profile_XX = run_login_query(db_name, table_name, 'dummy-XX', 'XXXXXXX')

# The following two lines indirectly mess up with the dummy-30 user
failed_30 = 8           # <= This is the initial value of the failed-count that will be set to the dummy-30 Profile
update_failed_attempts(db_name, table_name, 'dummy-30', failed_30 ) # <= failed_30 Is the failed attempts number we FORCE on the dummy-30 Profile
run_login_query(db_name, table_name, 'dummy-30', 'wrong-password') # <= You can Cut and Paste this line a few times to see how incorrect login affects the failed count

# Now we try login with the CORRECT password - see the effects in the database viewer and output below
user_profile_30 = run_login_query(db_name, table_name, 'dummy-30', '7777777')

print(user_profile_1)
print(user_profile_10)
print(user_profile_20)
print(user_profile_30)
print(user_profile_XX)

Rows:
[(1, 'dummy-1', 0)]
	  _______________________________________________
	 	 id  	 user_name  	 failed_count 	
	  _______________________________________________
	 	 1  	 dummy-1  	 0 	

The query constructed:
	 UPDATE OR IGNORE [Users_Topic_1a]   SET failed_count = :value         WHERE user_name= :user          

Rows:
[(10, 'dummy-10', 0)]
	  _______________________________________________
	 	 id  	 user_name  	 failed_count 	
	  _______________________________________________
	 	 10  	 dummy-10  	 0 	

The query constructed:
	 UPDATE OR IGNORE [Users_Topic_1a]   SET failed_count = :value         WHERE user_name= :user          

Rows:
[(20, 'dummy-20', 0)]
	  _______________________________________________
	 	 id  	 user_name  	 failed_count 	
	  _______________________________________________
	 	 20  	 dummy-20  	 0 	

The query constructed:
	 UPDATE OR IGNORE [Users_Topic_1a]   SET failed_count = :value         WHERE user_name= :user          

Rows:
[]

The query constructed: